Готово! Ниже — **быстрый универсальный toolkit для CLIP** (загрузка, препроцесс, эмбеддинги, zero-shot классификация, поиск по сходству, батчи, кэш, фолбэки).
Формат: **код + короткие пояснения в HTML-вставках**. Хочешь — просто удали HTML, останется чистый Python.

---

## 🚀 Быстрый старт

```python
# pip install torch torchvision pillow
# pip install open-clip-torch  # (желательно) современная реализация
# pip install ftfy regex tqdm   # опционально, если используешь openai/clip

```

---

## 0) Импорты и устройство

```python
import os, math, time, pathlib, warnings
from typing import List, Tuple, Dict, Optional

import numpy as np
from PIL import Image

import torch
from torch import nn
from torchvision import transforms

warnings.filterwarnings("ignore")

def get_device():
    if torch.cuda.is_available(): 
        return torch.device("cuda")
    if hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
        return torch.device("mps")  # Apple Silicon
    return torch.device("cpu")

DEVICE = get_device()
print("Device:", DEVICE)
```

```markdown
<div>
<b>Зачем:</b> выбираем доступный девайс. На соревнованиях без GPU всё работает, просто медленнее.<br>
<b>Совет:</b> для больших батчей и ускорения на GPU используем <code>float16</code> + <code>torch.autocast</code>.
</div>
```

---

## 1) Загрузка CLIP (open-clip → fallback на openai/clip)

```python
class CLIPWrapper:
    """
    Универсальная обёртка: сначала пытается загрузить open-clip,
    если не получилось — пробует openai/clip.
    """
    def __init__(self, model_name: str = "ViT-B-32", pretrained: str = "laion2b_s34b_b79k"):
        self.model_name = model_name
        self.pretrained = pretrained
        self.backend = None  # "open_clip" | "openai_clip"
        self.model = None
        self.preprocess = None
        self.tokenize = None
        self.image_mean = [0.48145466, 0.4578275, 0.40821073]
        self.image_std  = [0.26862954, 0.26130258, 0.27577711]
        self._load()

    def _load(self):
        # try open-clip first
        try:
            import open_clip
            self.model, _, self.preprocess = open_clip.create_model_and_transforms(
                self.model_name, pretrained=self.pretrained, device=DEVICE
            )
            self.model.eval().to(DEVICE)
            self.tokenize = open_clip.get_tokenizer(self.model_name)
            self.backend = "open_clip"
            return
        except Exception as e:
            print("[warn] open-clip не удалось:", e)

        # fallback: openai/clip
        try:
            import clip  # pip install git+https://github.com/openai/clip.git  (или pypi: clip-anytorch)
            self.model, self.preprocess = clip.load(self.model_name, device=DEVICE, jit=False)
            self.model.eval().to(DEVICE)
            self.tokenize = clip.tokenize
            self.backend = "openai_clip"
        except Exception as e:
            raise RuntimeError(
                "Не удалось загрузить CLIP ни через open-clip, ни через openai/clip. "
                "Проверь зависимости и веса."
            ) from e

    def build_image_transform(self, size: int = 224):
        # на случай, если хочешь свой трансформ (быстрее, чем PIL default)
        return transforms.Compose([
            transforms.Resize((size, size), interpolation=Image.BICUBIC),
            transforms.ToTensor(),
            transforms.Normalize(mean=self.image_mean, std=self.image_std),
        ])

    @torch.no_grad()
    def encode_images(self, images: List[Image.Image], batch_size: int = 32, fp16: bool = True) -> np.ndarray:
        """
        Возвращает L2-нормированные эмбеддинги изображений: [N, D]
        """
        embs = []
        tfm = self.preprocess  # готовый препроцесс от модели (часто fastest)
        # или кастомный: tfm = self.build_image_transform(224)

        use_autocast = fp16 and (DEVICE.type == "cuda")
        for i in range(0, len(images), batch_size):
            batch = images[i:i+batch_size]
            x = torch.stack([tfm(im) for im in batch]).to(DEVICE, non_blocking=True)
            with torch.autocast(device_type="cuda", dtype=torch.float16, enabled=use_autocast):
                feats = self.model.encode_image(x)
            feats = feats / feats.norm(dim=-1, keepdim=True)
            embs.append(feats.detach().float().cpu().numpy())
        return np.concatenate(embs, axis=0) if embs else np.zeros((0, 512), dtype=np.float32)

    @torch.no_grad()
    def encode_texts(self, texts: List[str], batch_size: int = 64, fp16: bool = True) -> np.ndarray:
        """
        Возвращает L2-нормированные эмбеддинги текстов: [M, D]
        """
        embs = []
        use_autocast = fp16 and (DEVICE.type == "cuda")
        for i in range(0, len(texts), batch_size):
            batch = texts[i:i+batch_size]
            tok = self.tokenize(batch).to(DEVICE, non_blocking=True)
            with torch.autocast(device_type="cuda", dtype=torch.float16, enabled=use_autocast):
                feats = self.model.encode_text(tok)
            feats = feats / feats.norm(dim=-1, keepdim=True)
            embs.append(feats.detach().float().cpu().numpy())
        return np.concatenate(embs, axis=0) if embs else np.zeros((0, 512), dtype=np.float32)

    @torch.no_grad()
    def zero_shot(self, images: List[Image.Image], classnames: List[str], templates: Optional[List[str]] = None,
                  batch_size: int = 32, fp16: bool = True) -> np.ndarray:
        """
        Zero-shot классификация: возвращает вероятности [N, C].
        Шаблоны (prompt ensembling) повышают точность.
        """
        if templates is None:
            templates = [
                "a photo of a {}",
                "a close-up photo of a {}",
                "a bright photo of a {}",
                "a dark photo of a {}",
                "a cropped photo of a {}",
                "a photo of the {}",
            ]
        # построим текстовый протокол (усреднение по шаблонам)
        text_embs_all = []
        for t in templates:
            texts = [t.format(c) for c in classnames]
            text_embs_all.append(self.encode_texts(texts, batch_size=batch_size, fp16=fp16))
        text_embs = np.stack(text_embs_all, axis=0).mean(axis=0)  # [C, D]
        text_embs = text_embs / np.linalg.norm(text_embs, axis=1, keepdims=True)

        # эмбеддинги изображений
        img_embs = self.encode_images(images, batch_size=batch_size, fp16=fp16)  # [N, D]
        # логиты ~ косинус*scale (обычно 100)
        logits = (img_embs @ text_embs.T) * 100.0
        # stable softmax
        logits -= logits.max(axis=1, keepdims=True)
        probs = np.exp(logits)
        probs /= probs.sum(axis=1, keepdims=True)
        return probs  # [N, C]
```

```markdown
<div>
<b>Почему так:</b> <code>open-clip</code> — более свежая и удобная реализация; если её нет — падаем на <code>openai/clip</code>.<br>
<b>Нормализация:</b> эмбеддинги L2-нормируются — косинус ≈ скалярное произведение.<br>
<b>Шаблоны:</b> prompt ensembling часто даёт +1–5% точности в zero-shot.
</div>
```

---

## 2) Утилиты загрузки данных

```python
def load_image(path: str, mode: str = "RGB") -> Image.Image:
    im = Image.open(path)
    return im.convert(mode) if mode else im

def load_images_from_paths(paths: List[str], mode: str = "RGB") -> List[Image.Image]:
    out=[]
    for p in paths:
        try: out.append(load_image(p, mode))
        except Exception as e: print("[warn] пропускаю", p, e)
    return out

def list_images(folder: str, exts=(".jpg",".jpeg",".png",".bmp",".webp")) -> List[str]:
    p = pathlib.Path(folder)
    files = [str(x) for x in p.rglob("*") if x.suffix.lower() in exts]
    return sorted(files)
```

```markdown
<div>
<b>Совет:</b> не загружай сразу десятки тысяч картинок в память. Делай батчи (см. <code>encode_images</code>) и/или сохраняй эмбеддинги на диск (.npy).
</div>
```

---

## 3) Быстрый пример: zero-shot на папке с классами

**Структура:**

```
dataset/
  cat/  (картинки котов)
  dog/  (картинки собак)
```

```python
def infer_classes_from_subdirs(root: str) -> Tuple[List[str], Dict[int, str], Dict[str, List[str]]]:
    """
    сбор: classnames, индекс->класс, класс->список путей
    """
    root = pathlib.Path(root)
    classnames = sorted([d.name for d in root.iterdir() if d.is_dir()])
    idx2cls = {i:c for i, c in enumerate(classnames)}
    cls2paths = {c: list_images(str(root / c)) for c in classnames}
    return classnames, idx2cls, cls2paths

# Демонстрация:
# root = "dataset"
# classnames, idx2cls, cls2paths = infer_classes_from_subdirs(root)
# print(classnames[:5], "…")
```

```python
def demo_zero_shot_folder(root: str, model_name="ViT-B-32", pretrained="laion2b_s34b_b79k",
                          max_per_class: int = 8):
    classnames, idx2cls, cls2paths = infer_classes_from_subdirs(root)
    clip = CLIPWrapper(model_name=model_name, pretrained=pretrained)
    templates = [
        "a photo of a {}",
        "a close-up photo of a {}",
        "a cropped photo of a {}",
        "a bright photo of a {}",
        "a dark photo of a {}",
    ]
    # соберём мини-набор изображений
    imgs, gts = [], []
    for c in classnames:
        paths = cls2paths[c][:max_per_class]
        imgs.extend(load_images_from_paths(paths, "RGB"))
        gts.extend([c]*len(paths))

    probs = clip.zero_shot(imgs, classnames, templates=templates, batch_size=32, fp16=True)  # [N, C]
    pred_idx = probs.argmax(axis=1)
    preds = [classnames[i] for i in pred_idx]

    acc = (np.array(preds) == np.array(gts)).mean() if len(gts) else 0.0
    print(f"Zero-shot accuracy (subset): {acc:.4f} on {len(gts)} images")

    # покажем топ-3 для первых 3 картинок
    for i in range(min(3, len(imgs))):
        order = np.argsort(-probs[i])[:3]
        print(f"[{i}] GT={gts[i]} | top3:", [(classnames[j], float(probs[i][j])) for j in order])

# Пример:
# demo_zero_shot_folder("dataset")
```

```markdown
<div>
<b>Где полезно:</b> когда нет размеченного train, а классы известны по названию. Типично для быстрых прототипов/проверок гипотез.<br>
<b>Качество:</b> сильно зависит от названий классов и качества шаблонов (prompt engineering).
</div>
```

---

## 4) Поиск по сходству (image→text, text→image)

```python
def cosine_sim(a: np.ndarray, b: np.ndarray) -> np.ndarray:
    # предполагаем L2-нормировку; иначе нормализуй: a/=||a||, b/=||b||
    return a @ b.T

def topk_sim(sims: np.ndarray, k: int = 5) -> List[List[Tuple[int, float]]]:
    """
    sims: [N, M] — сходства между N запросами и M кандидатами
    Возвращает top-k индексов и значений для каждой строки.
    """
    N, M = sims.shape
    out = []
    for i in range(N):
        idx = np.argpartition(-sims[i], k)[:k]
        idx = idx[np.argsort(-sims[i, idx])]
        out.append([(int(j), float(sims[i, j])) for j in idx])
    return out

def demo_retrieval(images: List[Image.Image], texts: List[str], model_name="ViT-B-32", pretrained="laion2b_s34b_b79k"):
    clip = CLIPWrapper(model_name=model_name, pretrained=pretrained)
    img_embs  = clip.encode_images(images, batch_size=32, fp16=True)   # [N, D]
    text_embs = clip.encode_texts(texts,  batch_size=64, fp16=True)    # [M, D]
    sims = cosine_sim(img_embs, text_embs)  # [N, M]

    # image -> top texts
    top_texts = topk_sim(sims, k=3)
    # text -> top images (по столбцам)
    top_images = topk_sim(sims.T, k=3)

    return top_texts, top_images

# Пример:
# imgs = load_images_from_paths(list_images("dataset/cat")[:5])
# texts = ["a photo of a cat", "a photo of a dog", "a street", "a mountain"]
# it, ti = demo_retrieval(imgs, texts)
# print("img->text:", it[0])
# print("text->img:", ti[0])
```

```markdown
<div>
<b>Зачем:</b> быстрый ретривл (поиск) — основа мультимодальных задач: поисковики по галерее, подбор подписей, проверка качества эмбеддингов.<br>
<b>Нормализация:</b> здесь мы уже вернули L2-нормированные эмбеддинги — косинус = скалярное произведение.
</div>
```

---

## 5) Кэширование эмбеддингов на диск (ускоряет эксперименты)

```python
def save_embeddings(path: str, array: np.ndarray):
    os.makedirs(os.path.dirname(path) or ".", exist_ok=True)
    np.save(path, array)

def load_embeddings(path: str) -> np.ndarray:
    return np.load(path)

# Пример:
# img_embs = clip.encode_images(imgs)
# save_embeddings("cache/img_embs.npy", img_embs)
# img_embs = load_embeddings("cache/img_embs.npy")
```

```markdown
<div>
<b>Зачем:</b> чтобы не кодировать изображения/тексты заново при каждом запуске (экономия минут/часов).<br>
<b>Совет:</b> храни 1) список путей; 2) массив эмбеддингов; 3) версию модели (чтобы не перепутать).
</div>
```

---

## 6) Мини-CLI/скрипт (самый быстрый сценарий)

```python
if __name__ == "__main__":
    # 1) модель
    clip = CLIPWrapper(model_name="ViT-B-32", pretrained="laion2b_s34b_b79k")

    # 2) картинки
    img_paths = list_images("dataset/cat")[:4] + list_images("dataset/dog")[:4]
    images = load_images_from_paths(img_paths)

    # 3) zero-shot
    classes = ["cat", "dog"]
    probs = clip.zero_shot(images, classes)  # [N, C]
    preds = [classes[i] for i in probs.argmax(axis=1)]
    print("Zero-shot preds:", preds[:8])

    # 4) ретривл: image->text
    texts = [f"a photo of a {c}" for c in classes]
    img_embs  = clip.encode_images(images)
    text_embs = clip.encode_texts(texts)
    sims = cosine_sim(img_embs, text_embs)
    print("Top text for first image:", topk_sim(sims, 1)[0][0])
```

---

## ⚠️ Типичные ошибки и рекомендации

```markdown
<div>
<ul>
<li><b>Нет интернета на кластере</b>: подготовь веса заранее (папка с моделью). В <code>open_clip.create_model_and_transforms</code> укажи путь к локальным весам.</li>
<li><b>CUDA OOM</b>: уменьшай batch_size, включай fp16 (autocast), используй <code>with torch.no_grad()</code>.</li>
<li><b>Плохое качество zero-shot</b>: сделай prompt ensembling (несколько шаблонов), уточняй формулировки классов (например, "German shepherd dog" вместо "dog").</li>
<li><b>Слишком много изображений</b>: кодируй партиями и сразу сохраняй эмбеддинги на диск (.npy).</li>
<li><b>Разный размер входа</b>: всегда нормализуй препроцессом, который вернул CLIP (он быстрый и совместимый).</li>
<li><b>Валидация</b>: zero-shot — не магия. Для итоговой модели лучше дообучение или хотя бы адаптация с несколькими размеченными примерами (few-shot + linear probe).</li>
</ul>
</div>
```

---

## 🧩 Быстрый few-shot (linear probe поверх эмбеддингов)

```python
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

def linear_probe(train_imgs: List[Image.Image], y: List[int],
                 val_imgs: List[Image.Image], y_val: List[int],
                 model_name="ViT-B-32", pretrained="laion2b_s34b_b79k") -> float:
    clip = CLIPWrapper(model_name, pretrained)
    X_tr = clip.encode_images(train_imgs, batch_size=64, fp16=True)
    X_va = clip.encode_images(val_imgs,   batch_size=64, fp16=True)

    clf = LogisticRegression(max_iter=2000, n_jobs=4)
    clf.fit(X_tr, y)
    pred = clf.predict(X_va)
    return accuracy_score(y_val, pred)

# Пример:
# acc = linear_probe(train_imgs, y, val_imgs, y_val)
# print("Few-shot accuracy:", acc)
```

```markdown
<div>
<b>Когда нужно:</b> если zero-shot слабоват, а у тебя есть немного размеченных данных. Linear probe часто даёт заметный прирост за считанные минуты.
</div>
```

---

### Готово!

* Это **универсальный, быстрый** CLIP-toolkit: загрузка, препроцесс, эмбеддинги, zero-shot, ретривл, кэш, few-shot.
* Можно вставить целиком в один файл и сразу работать.
* Нужна версия без комментариев/HTML — напишу чистый `.py`.
